In [7]:
import concurrent.futures
from scripts import request
from scripts import table
from scripts import utils

In [8]:
def run(connection_data: dict, messages_file_paths: dict, articles_tables: dict, output_path: str, num_threads: int):
    clients = [request.init_client(connection_data) for _ in range(num_threads)]

    original_prompt = f'''
        SYSTEM 1: \n{request.read_file(messages_file_paths['system_1'])}\n\n
        USER 1:\n{request.read_file(messages_file_paths['user_1'])}\n\n
        ASSISTANT:\n{request.read_file(messages_file_paths['assistant'])}\n\n
        USER 2:\n{request.read_file(messages_file_paths['user_2'])}\n\n
        SYSTEM 2:\n{request.read_file(messages_file_paths['system_2'])}\n\n
    '''

    with open(output_path + '/prompt.txt', "w") as text_file:
        text_file.write(original_prompt)

    progress = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        for article_id, article_tables in articles_tables.items():
            for index, article_table in enumerate(article_tables):
                if not article_table['processed']:
                    executor.submit(
                        request.extract_claims,
                        clients[progress % num_threads],
                        article_table,
                        f"{article_id}_{index}",
                        messages_file_paths,
                        output_path
                    )

                    progress += 1

    for client in clients:
        client.close()

    return

In [9]:
connection_infos = request.extract_infos('private.json')

msgs_base_path = 'messages/CS'

msgs_file_paths = {
    'system_1':  f'{msgs_base_path}/system_1.txt',
    'system_2':  f'{msgs_base_path}/system_2.txt',
    'user_1':    f'{msgs_base_path}/user_1.txt',
    'user_2':    f'{msgs_base_path}/user_2.txt',
    'assistant': f'{msgs_base_path}/assistant.txt'
}

output_folder = 'experiments/output/cs_main/2'
utils.check_path(output_folder)

tables_file_path = 'experiments/extracted_tables/cs_tables.json'

table.check_processed_tables(tables_file_path, output_folder + "/answers")

In [12]:
tables = table.load_tables_from_json(tables_file_path)
run(connection_infos, msgs_file_paths, tables, output_folder, 1)

	 Saved prompt at: experiments/output/cs_main/2/prompts\2307.01231_3.txt
Sending request for: [2307.01231_3]
<{<Method, DeepMatcher (15)>, <Dataset, D_{s1}>}, F1, 98.65>

<{<Method, DeepMatcher (15)>, <Dataset, D_{s2}>}, F1, 95.50>

<{<Method, DeepMatcher (15)>, <Dataset, D_{s3}>}, F1, 88.46>

<{<Method, DeepMatcher (15)>, <Dataset, D_{s4}>}, F1, 69.66>

<{<Method, DeepMatcher (15)>, <Dataset, D_{s5}>}, F1, 75.86>

<{<Method, DeepMatcher (15)>, <Dataset, D_{s6}>}, F1, 65.98>

<{<Method, DeepMatcher (15)>, <Dataset, D_{s7}>}, F1, 95.45>

<{<Method, DeepMatcher (15)>, <Dataset, D_{d1}>}, F1, 96.63>

<{<Method, DeepMatcher (15)>, <Dataset, D_{d2}>}, F1, 93.07>

<{<Method, DeepMatcher (15)>, <Dataset, D_{d3}>}, F1, 75.00>

<{<Method, DeepMatcher (15)>, <Dataset, D_{d4}>}, F1, 46.56>

<{<Method, DeepMatcher (15)>, <Dataset, D_{t1}>}, F1, 68.53>

<{<Method, DeepMatcher (15)>, <Dataset, D_{t2}>}, F1, 94.04>

<{<Method, DeepMatcher (40)>, <Dataset, D_{s1}>}, F1, 98.76>

<{<Method, DeepMatcher 

In [11]:
table.check_processed_tables(tables_file_path, output_folder + "/answers")